In [107]:
import requests
from bs4 import BeautifulSoup as bs
import pymysql as mysql
import pandas as pd
%run secrets
def scrape_kenpom():
	bball_data = []
	cols_type = ['Year int', 'Rank int', 'Team varchar(32)', 'Conf varchar(10)', 'Win_Loss varchar(10)', 'Eff_Margin float', 'Off_Eff float', 'Rank_Off_Eff int', 'Def_Eff float', 'Rank_Def_Eff int', 'Tempo float', 'Rank_Tempo int', 'Luck float', 'Rank_Luck int', 'sos float', 'Rank_SOS int', 'Avg_Opp_Off_Eff float', 'Rank_Off_Eff int', 'Avg_Opp_Def_Eff float', 'Rank_Opp_Def_Eff int', 'Non_Conf_SOS float', 'Rank_NonConf_SOS int']
	cols = ['Year', 'Rank', 'Team', 'Conf', 'Win_Loss', 'Eff_Margin', 'Off_Eff', 'Rank_Off_Eff', 'Def_Eff', 'Rank_Def_Eff', 'Tempo', 'Rank_Tempo', 'Luck', 'Rank_Luck', 'SOS', 'Rank_SOS', 'Avg_Opp_Off_Eff', 'Rank_Off_Eff', 'Avg_Opp_Def_Eff', 'Rank_Opp_Def_Eff', 'Non_Conf_SOS', 'Rank_NonConf_SOS']
	website = 'https://kenpom.com/'
	r = requests.get(website)
	links = bs(r.content, 'lxml').find('div', {'id': 'content-header'}).find('span', {'class': 'rank'}).findAll('a')
	links = links[1:-1]
	url_list = []
	for link in links:
		url_list.append(website + link.attrs.get('href'))
	url_list.append(website + 'index.php')

	for url in url_list:
		r = requests.get(url)
		soup = bs(r.content, 'lxml')
		year = soup.find('div', {'id': 'content-header'}).find('h2').text[:4]
		data_tables = soup.find('table', {'id': 'ratings-table'}).findAll('tbody')

		for table in data_tables:
			teams = table.findAll('tr')

			for team in teams:
				stats = team.findAll('td')
				team_data = [year]

				for stat in stats:
					stat_links = stat.findAll('a')
					if len(stat_links):
						stat = stat_links[0]
					stat_text = stat.text.replace('+', '')
					team_data.append(stat_text)

				if len(team_data) > 1:
					bball_data.append(team_data)
		
		print('Scrape complete for ' + year)
	df = pd.DataFrame(bball_data, columns=cols)
	
	return df, cols, cols_type


df, cols, col_types = scrape_kenpom()

df.to_csv('../data/Master.csv', index=False)

Scrape complete for 2002
Scrape complete for 2003
Scrape complete for 2004
Scrape complete for 2005
Scrape complete for 2006
Scrape complete for 2007
Scrape complete for 2008
Scrape complete for 2009
Scrape complete for 2010
Scrape complete for 2011
Scrape complete for 2012
Scrape complete for 2013
Scrape complete for 2014
Scrape complete for 2015
Scrape complete for 2016
Scrape complete for 2017
Scrape complete for 2018
Scrape complete for 2019
Scrape complete for 2020
Scrape complete for 2021
Scrape complete for 2022


In [171]:
import os
import csv
import codecs
import pymysql


connection = pymysql.connect(
    host=HOST,
	user=USER,
	password=PASSWORD,
	db=DATABASE
	)

cursor = connection.cursor()

directory_str = "../data/"
for sub_dirs, dirs, files in os.walk(directory_str):
    for current_file in files:
        if current_file.endswith("csv"):
            file_path = directory_str + current_file
            table_name = current_file[:-4]

            if table_name =='Master':
                header = ['Year int', 'Team_Rank int','Team varchar(32)','Conf varchar(10)','Win_Loss varchar(10)','Eff_Margin float','Off_Eff float','Rank_Off_Eff int','Def_Eff float','Rank_Def_Eff int','Tempo float','Rank_Tempo int','Luck float','Rank_Luck int','SOS float','Rank_SOS int','Avg_Opp_Off_Eff float','Rank_Opp_Off_Eff int','Avg_Opp_Def_Eff float','Rank_Opp_Def_Eff int','Non_Conf_SOS float','Rank_NonConf_SOS int']
            elif table_name == 'Results':
                header = ['Year int', 'Day int', 'WinningID int', 'Winning_Score int', 'LosingID int', 'Losing_Score int', 'Winner_Location varchar(32)', 'Overtime int']
            elif table_name == 'Seeds':
                header = ['Year int', 'Seed varchar(6)', 'TeamID int']
            elif table_name == 'Slots':
                header = ['Year int', 'Slot varchar(6)', 'Strong_Seed varchar(6)', 'Weak_Seed varchar(6)']
            elif table_name == 'Teams':
                header = ['Team_ID int', 'Table_Name varchar(32)', 'FirstD1Season int', 'LastD1Season int']
            elif table_name == 'TeamSpellings':
                header = ['TeamSpelling varchar(32)', 'TeamID int']

            cursor.execute("SHOW TABLES LIKE '{}'".format(table_name))
            result = cursor.fetchone()
            print(result)
            if result == None:
                cursor.execute("CREATE TABLE {} ({})".format(table_name, ', '.join(header)))
                connection.commit()
            else:
                cursor.execute("DROP TABLE {}".format(table_name))
                cursor.execute("CREATE TABLE {} ({})".format(table_name, ', '.join(header)))
                connection.commit()

            with codecs.open(file_path, 'r',  encoding='utf-8', errors='ignore') as raw_file:
                open_csv = csv.reader(raw_file)
                columns = next(open_csv)

                for data in open_csv:
                    formatted_row=[]
                    for item in data:
                        if type(item) == str:
                            formatted_row.append('"' + item + '"')
                        else:
                            formatted_row.append(item)
                    query = 'insert into {} values ({})'.format(table_name, ', '.join(formatted_row))
                    cursor.execute(query)
                    print(query)
                connection.commit()
                print(query)
        else:
            continue

('Teams',)
insert into Teams values ("1101", "Abilene Chr", "2014", "2018")
insert into Teams values ("1102", "Air Force", "1985", "2018")
insert into Teams values ("1103", "Akron", "1985", "2018")
insert into Teams values ("1104", "Alabama", "1985", "2018")
insert into Teams values ("1105", "Alabama A&M", "2000", "2018")
insert into Teams values ("1106", "Alabama St", "1985", "2018")
insert into Teams values ("1107", "Albany NY", "2000", "2018")
insert into Teams values ("1108", "Alcorn St", "1985", "2018")
insert into Teams values ("1109", "Alliant Intl", "1985", "1991")
insert into Teams values ("1110", "American Univ", "1985", "2018")
insert into Teams values ("1111", "Appalachian St", "1985", "2018")
insert into Teams values ("1112", "Arizona", "1985", "2018")
insert into Teams values ("1113", "Arizona St", "1985", "2018")
insert into Teams values ("1114", "Ark Little Rock", "1985", "2018")
insert into Teams values ("1115", "Ark Pine Bluff", "1999", "2018")
insert into Teams value